# Imports

In [32]:
import numpy as np
import geopandas as gpd
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import libpysal.weights as weights
import pysal.explore as esda

# Load data

In [33]:
raw_df = gpd.read_file("datasets/5_split/df_fe.gpkg")

# Separate features

In [34]:
features = raw_df.copy()
features["x_coord"] = features["geometry"].centroid.x
features["y_coord"] = features["geometry"].centroid.y
features = features.drop(columns=["geometry"])
predictor_cols = ["greenspace_proportion", "imd", "f_m_ratio", "mean_age"]
labels = features.pop("very_good_health")

# Get model inputs

In [35]:
coords = np.array(list(zip(features["x_coord"], features["y_coord"])))
target = labels.values.reshape((-1, 1))
predictors = np.hstack(
    [features[col].values.reshape((-1, 1)) for col in predictor_cols]
)
bandwidth = Sel_BW(coords, target, predictors, kernel="bisquare").search(
    criterion="AICc"
)

# Build model

In [36]:
# Build model
model = GWR(
    coords,
    target,
    predictors,
    bw=bandwidth,
    kernel="bisquare",
)

# Fit model
model.fit()

# Calculate spatial autocorrelation of residuals

In [37]:
predictions = model.predict(coords, predictors).predy.flatten()
residuals = labels - predictions
features["residuals"] = residuals
w = weights.KNN.from_dataframe(raw_df, k=8)
moran = esda.esda.Moran(features["residuals"], w)

In [38]:
moran.I

np.float64(0.038629578211883904)